# I. Building generic model

## 0. Import libraries

In [1]:
import time
import copy
import numpy as np
from tqdm import trange
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import warnings
warnings.filterwarnings(action="ignore")
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
CUDA = torch.cuda.is_available()
CUDA_DEVICE = 0
if CUDA: device='cuda'
else: device='cpu'
device

'cpu'

## 1. Neural networks

### 1-0 Functions

In [3]:
def activation(Tensor): 
    return torch.tanh(Tensor)

def dU(k,position): 
    return k*position

def time_evolution(ntraj,tlength,dt,nptl,gamma,gammaR,DT,DR,k,r,v,force):
    State_tensor=torch.zeros(size=(ntraj,tlength+1,4*nptl)); State_list=[]; 
    Deformed_State_tensor=torch.zeros(size=(ntraj,tlength+1,1+4*nptl)); Deformed_State_list=[]
    Noise_tensor=torch.normal(mean=0,std=1/np.sqrt(dt),size=(ntraj,tlength+1,4*nptl)); Noise_list=[]
    for traj in range(ntraj):
        r_t=r[traj,-1,:]; v_t=v[traj,-1,:]
        State_tensor[traj,0,:2*nptl]=r_t; State_tensor[traj,0,2*nptl:]=v_t
        Deformed_State_tensor[traj,0,0]=0; Deformed_State_tensor[traj,0,1:]=State_tensor[traj,0,:]
        for t in range(tlength):
            r_t += dt*(v_t-1/gamma*dU(k,r_t)+force[traj,t,:2*nptl]+np.sqrt(2*DT)*Noise_tensor[traj,t,:2*nptl])
            v_t += dt*(-1/gammaR*v_t+force[traj,t,2*nptl:]+np.sqrt(2*DR)*Noise_tensor[traj,t,2*nptl:])
            State_tensor[traj,t+1,:2*nptl]=r_t; State_tensor[traj,t+1,2*nptl:]=v_t
            Deformed_State_tensor[traj,t+1,0]=(t+1)*dt; Deformed_State_tensor[traj,t+1,1:]=State_tensor[traj,t+1,:]    
        State_list.append(State_tensor[traj,...].numpy())
        Deformed_State_list.append(Deformed_State_tensor[traj].numpy())
        Noise_list.append(Noise_tensor[traj,...].numpy())
    return State_tensor, torch.tensor(State_list), torch.tensor(Deformed_State_list), torch.tensor(Noise_list)

### 1-1. Ritz Block

In [4]:
class RitzBlock(nn.Module):
    def __init__(self, n_hid):
        super(RitzBlock, self).__init__()
        self.n_hid = n_hid
        ##Layer 1
        self.Weight_layer1 = nn.Parameter(torch.rand(size=(n_hid,n_hid),requires_grad=True)).to(device)
        self.Bias_layer1 = nn.Parameter(torch.rand(size=(1,n_hid),requires_grad=True)).to(device)
        ##Layer 2
        self.Weight_layer2 = nn.Parameter(torch.rand(size=(n_hid,n_hid),requires_grad=True)).to(device)
        self.Bias_layer2 = nn.Parameter(torch.rand(size=(1,n_hid),requires_grad=True)).to(device)

    #Activation function
    def layer1(self,Tensor): return activation(torch.matmul(Tensor,self.Weight_layer1)+self.Bias_layer1)
    def layer2(self,Tensor): return activation(torch.matmul(Tensor,self.Weight_layer2)+self.Bias_layer2)
    
    #Forward process
    def forward(self,Tensor):
        h1=self.layer1(Tensor).to(device)
        h2=self.layer2(h1).to(device)
        return h2+Tensor #return.shape = (tlength+1,nhid)

In [5]:
class PreModel(nn.Module):
    def __init__(self,n_hid):
        super(PreModel,self).__init__()
        self.n_hid=n_hid
        ##premodel parameters
        self.pre_Weight=nn.Parameter(torch.rand(size=(4*nptl+1,n_hid),requires_grad=True)).to(device)
        self.pre_Bias=nn.Parameter(torch.rand(size=(1,n_hid),requires_grad=True)).to(device)
    
    def forward(self,Tensor):
        if n_hid>4*nptl+1: 
            pad=torch.nn.ZeroPad2d((0,n_hid-4*nptl-1,0,0))
            output=pad(Tensor)
        else:
            output=activation(torch.matmul(Tensor,self.pre_Weight)+self.pre_Bias)
        return output.to(device)

### 1-2. Neural networks

In [11]:
class NeuralNetworks(nn.Module):
    def __init__(self,n_block,n_hid):
        super(NeuralNetworks, self).__init__()
        self.n_blocks=n_block
        self.n_hid=n_hid
        self.nn_Weight=nn.Parameter(torch.rand(size=(n_hid,4*nptl),requires_grad=True)).to(device)
        self.nn_Bias=nn.Parameter(torch.rand(size=(1,4*nptl),requires_grad=True)).to(device)        
        self.premodel=PreModel(self.n_hid)
        self.Ritzblocks=nn.ModuleList()
        for _ in range(self.n_blocks): self.Ritzblocks.append(RitzBlock(self.n_hid))
    
    def Observable(self,Tensor,noise):
        Noise=dt*noise[...,:-1,:2*nptl]; position=Tensor[...,:-1,:2*nptl]
        velocity=Tensor[...,:-1,2*nptl:]; velocity1=Tensor[...,1:,2*nptl:]
        iteration=Noise.shape[0] #batch size
        first=dt*torch.bmm(velocity,torch.permute(velocity,(0,2,1)))
        second=dt*torch.bmm(velocity,torch.permute(dU(k,position),(0,2,1)))
        third=torch.bmm(velocity+velocity1,torch.permute(Noise,(0,2,1)))/2        
        Obs=torch.zeros(iteration) #zero matrix
        for n in range(iteration):
            Obs[n]=torch.trace(gamma*(first[n,...]-second[n,...]+np.sqrt(2*DT)*third[n,...])/(dt*tlength))
        return torch.mean(Obs)

    def Girsanov_weight(self,force):        
        #force.shape=(ntraj,tlength+1,4*nptl)
        F_r=force[...,:-1,:2*nptl]
        F_v=force[...,:-1,2*nptl:]
        iteration=force.shape[0]
        position_part=dt*torch.bmm(F_r,torch.permute(F_r,(0,2,1)))
        velocity_part=dt*torch.bmm(F_v,torch.permute(F_v,(0,2,1)))
        Girsanov=torch.zeros(iteration)
        for n in range(iteration):
            Girsanov[n]=torch.trace(position_part[n,...]/DT+velocity_part[n,...]/DR)
        return -0.5*torch.mean(Girsanov)

    #def loss(self,Tensor,noise,force):
    #    L=Lambda*(dt*tlength)*self.Observable(Tensor,noise)+self.Girsanov_weight(force)
    #    return L

    def forward(self,Tensor):
        T=self.premodel(Tensor).to(device)
        for model in self.Ritzblocks:
            val=model(T); T=val
        del_u=torch.matmul(T,self.nn_Weight)+self.nn_Bias
        return del_u #variational force

### 1-3 Batch loader

In [12]:
class CustomDataset(Dataset): 
    def __init__(self, dataset):
        data_x = dataset
        self.x_data = data_x
#         self.y_data = data_y

    # 총 데이터의 개수를 리턴
    def __len__(self): 
        return len(self.x_data)
    
    # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, idx): 
        x = torch.FloatTensor(self.x_data[idx])
#         y = torch.FloatTensor([self.y_data[idx]])
        return x

def data_to_loader(fullconfigs):
    fulldata=CustomDataset(fullconfigs)
    full_dataset = fulldata
    full_loader = torch.utils.data.DataLoader(full_dataset,batch_size,shuffle=False)
    return full_loader

## 2. Solving SDE

In [13]:
def AOUP(Lambda,lr,epochs,k,n_hid,n_block,ntraj,tlength,dt,nptl,gamma,gammaR,DT,DR):
    NN=NeuralNetworks(n_block,n_hid)
    train_op=optim.SGD(NN.parameters(),lr,momentum=0.9)
    NN.train()
    train_loss_list=[]; 
    ActiveWork_list=[]; GirsanovWeight_list=[]
    
    #Initial condition
    r_0=torch.zeros(size=(ntraj,tlength+1,2*nptl)); v_0=torch.zeros_like(r_0)
    force=torch.zeros(size=(ntraj,tlength+1,4*nptl))
    
    for _ in trange(epochs,desc="Epoch"):
        train_loss_epoch=[]; ActiveWork_epoch=[]; GirsanovWeight_epoch=[]
        # 0. Generate trajectory using previously defined initial condition
        States,train,train_real,train_noise=time_evolution(ntraj,tlength,dt,nptl,gamma,gammaR,DT,DR,k,r_0,v_0,force)
        #train_real is list of data.shape=(tlength+1,1+4*nptl) <- 진짜 훈련에 사용됨 (loss는 다른 데이터로 계산)
        # 1. Update initial state
        r_0=States[...,:2*nptl];v_0=States[...,2*nptl:]
        #train data를 GPU에 올린다.        
        train_loader=data_to_loader(train); train_real_loader=data_to_loader(train_real); noise_loader=data_to_loader(train_noise)
        train_n_noise=list(zip(train_loader,train_real_loader,noise_loader))

        for _, (data_n_noise) in enumerate(train_n_noise):
            data=data_n_noise[0].to(device); data_input=data_n_noise[1].to(device); noise=data_n_noise[2].to(device)
            del_u=NN.forward(data_input)
            ActiveWork=NN.Observable(data,noise); GirsanovWeight=NN.Girsanov_weight(del_u)
            ActiveWork_epoch.append(ActiveWork.detach()); GirsanovWeight_epoch.append(GirsanovWeight.detach())
            train_loss=Lambda*dt*tlength*ActiveWork+GirsanovWeight
            train_loss_epoch.append(train_loss.item())
            train_op.zero_grad()
            train_loss.backward()
            train_op.step()
        
        train_loss_list.append(np.mean(train_loss_epoch));
        ActiveWork_list.append(np.mean(ActiveWork_epoch));
        GirsanovWeight_list.append(np.mean(GirsanovWeight_epoch))
    NN=NN.cpu()
    return train_loss_list, ActiveWork_list, GirsanovWeight_list

In [14]:
Lambda=-2.0
learning_rate=0.05
epochs=500
k=1
n_hid=1000; n_block=3
ntraj=50; tlength=1000; dt=0.001; nptl=1
gamma=1; gammaR=1
DT=1; DR=1
batch_size=10

In [15]:
loss,AW,GW=AOUP(Lambda,learning_rate,epochs,k,n_hid,n_block,ntraj,tlength,dt,nptl,gamma,gammaR,DT,DR)

Epoch:  78%|███████▊  | 388/500 [1:10:52<20:27, 10.96s/it]


KeyboardInterrupt: 